# Project References

The code of this project was mainly adapted from the tutorial *Combining Categorical and Numerical Features with Text in BERT*, by Chris McCormick, which can be found at https://mccormickml.com/2021/06/29/combining-categorical-numerical-features-with-bert/#24-bert-on-review-text-only. The method for saving the best model during training was also based on another tutorial from the same author called *BERT Fine-Tuning Tutorial with PyTorch*, which was made available at https://mccormickml.com/2019/07/22/BERT-fine-tuning/.

# Importing Libraries and Preparing the environment

## Mount Google Drive to this Notebook instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing Transformers and Spacy libraries

In [ ]:
!pip install -q transformers

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import statistics
import transformers
import torch
import time
import datetime
import random
import gc

from torch import cuda
from build_dataset import Corpus
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn import metrics
from sklearn.metrics import accuracy_score, cohen_kappa_score, mean_squared_error

## Configuring use of GPU


In [ ]:
# If there's a GPU available...
if cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


# Loading and Preprocessing Data

## Reading Corpus (Essay-Br)

### Essays

In [ ]:
c = Corpus()

train_df, validation_df, test_df = c.read_splits()

train_df.head()

,prompt,title,essay,c1,c2,c3,c4,c5,score
0,86,Bem está mental,[É notório que as redes sociais estão cada vez...,120,120,120,160,160,680
1,86,A questão das redes sociais na saúde mental da...,[Com o avanço das tecnologias ao redor do mund...,200,160,160,200,160,880
2,47,Democratização do ensino remoto no Brasil,[A declaração universal dos direitos humanos a...,160,160,160,160,160,800
3,89,"ZZZ (POR FAVOR, IGNOREM O TAMANHO, PRECISO DA ...","[Barão de Itararé, um dos criadores do jornali...",160,160,160,200,200,880
4,40,Um samba enredo que está por vir,"[No carnaval deste ano, a escola de samba Uniã...",160,160,120,200,200,840


### Prompts

In [ ]:
prompts_df = pd.read_csv("./extended-corpus/extended_prompts.csv")

prompts_df.head()

,id,title,description,category
0,0,Carnaval e apropriação cultural,"['No Carnaval de 2020, veio novamente à tona u...",sociedade e cultura
1,1,Qualificação e o futuro do emprego,['O número de pessoas desempregadas no mundo d...,economia
2,2,Supremo Tribunal Federal e opinião pública,"['Ao longo dos últimos dez anos, o papel do ST...",política
3,3,"Ciência, tecnologia e superação dos limites hu...","['Com o avanço da biotecnologia, da engenharia...",ciência e tecnologia
4,4,Um réu deve ou não ser preso após a condenação...,"['No início deste mês de novembro, o Supremo T...",sociedade e cultura


## Adjusting Dataframes

In [ ]:
competence = "c3"

def adjust_columns(dataframe, competence):
    dataframe["label"] = [label//40 for label in dataframe[competence]]
    new_df = dataframe[['essay', 'label']].copy()
    return new_df

preprocessed_train_df = adjust_columns(train_df, competence)
preprocessed_validation_df = adjust_columns(validation_df, competence)
preprocessed_test_df = adjust_columns(test_df, competence)

preprocessed_train_df.head()

,essay,label
0,[É notório que as redes sociais estão cada vez...,3
1,[Com o avanço das tecnologias ao redor do mund...,4
2,[A declaração universal dos direitos humanos a...,4
3,"[Barão de Itararé, um dos criadores do jornali...",4
4,"[No carnaval deste ano, a escola de samba Uniã...",3


# Preparing the Dataset and Dataloader

## Configuring variables

In [ ]:
max_len = 512
batch_size = 16
epochs = 6
learning_rate = 2e-5

checkpoint = "neuralmind/bert-base-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

## Creating the datasets for the neural network

### Tokenize function

In [ ]:
def create_tokenized_dataset(dataframe):
    # Tokenize all of the sentences and map the tokens to their word IDs.
    input_ids = []
    attention_masks = []

    print('Encoding all essays in the dataset...')

    # For every sentence...
    for essay in dataframe['essay']:
        encoded_dict = tokenizer.encode_plus(
                            " ".join(essay),                # Sentence to encode.
                            add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                            max_length = max_len,           # Pad & truncate all sentences.
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',          # Return pytorch tensors.
                    )

        # Add the encoded sentence to the list.
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(dataframe['label'].tolist())

    return TensorDataset(input_ids, attention_masks, labels)

### Create Tokenized Datasets

In [ ]:
train_dataset = create_tokenized_dataset(preprocessed_train_df)
val_dataset = create_tokenized_dataset(preprocessed_validation_df)
test_dataset = create_tokenized_dataset(preprocessed_test_df)

Encoding all essays in the dataset...
Encoding all essays in the dataset...
Encoding all essays in the dataset...


### Creating the dataloader for the neural network

In [ ]:
train_dataloader = DataLoader(
            train_dataset,                            # The training samples.
            sampler = RandomSampler(train_dataset),   # Select batches randomly
            batch_size = batch_size                   # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset,                              # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size                   # Evaluate with this batch size.
        )

# Fine Tuning

## Loading pretrained model

In [ ]:
class CustomModel(torch.nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained(checkpoint)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)

    def forward(self, ids, attention_mask):
        # Get pooled output from BERT layer
        _, output_1 = self.l1(ids, attention_mask, return_dict=False)
        # Apply dropout
        output_2 = self.l2(output_1)
        # Apply linear transformation
        output = self.l3(output_2)
        return output

## Defining auxiliary Variables, Functions and Classes

### Loss Function

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

### Accuracy function

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

### Time Format function

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

### Class to save best model

In [ ]:
class SaveBestModel:
    """
    Class to save the best model while training. If the current epoch's
    validation loss is less than the previous least less, then save the
    model state.
    """
    def __init__(
        self, model_i, best_valid_loss=float('inf')
    ):
        self.best_valid_loss = best_valid_loss
        self.model_i = model_i + 1

    def __call__(
        self, current_valid_loss,
        epoch, model, optimizer
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            path = f'./drive/MyDrive/TCC/code/model_base/best_model_{self.model_i}.pth'
            torch.save({
                'epoch': epoch+1,
                'valid_loss': current_valid_loss,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, path)

# Train, Eval and Test Loop

In [ ]:
models = 10
metrics = []

In [ ]:
for model_i in range(0, models):

    print("")
    print('=====================================')
    print('========== Model {:} / {:} =========='.format(model_i + 1, models))
    print('=====================================')

    # ========================================
    #               Setup
    # ========================================

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # Initialize SaveBestModel class
    save_best_model = SaveBestModel(model_i)

    # Instantiate model
    model = CustomModel()
    model.to(device)

    # Define optimizer
    optimizer = AdamW(model.parameters(),
              lr = learning_rate,
              eps = 1e-8
            )

    # Total number of training steps is [number of batches] x [number of epochs].
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)


    # For each epoch...
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 100 batches.
            if step % 100 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader.
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids
            #   [1]: attention masks
            #   [2]: labels
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because
            # accumulating the gradients is "convenient while training RNNs".
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()

            # Perform a forward pass (evaluate the model on this training batch).
            # In PyTorch, calling `model` will in turn call the model's `forward`
            # function and pass down the arguments.

            logits = model(b_input_ids,
                          attention_mask=b_input_mask)

            # Calculate Cross-Entropy Loss for this batch.
            loss = loss_fn(logits, b_labels)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value
            # from the tensor.
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader.
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids
            #   [1]: attention masks
            #   [2]: labels
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():

                # Forward pass, calculate logit predictions. The "logits" are the
                # output values prior to applying an activation function like the
                # softmax.
                logits = model(b_input_ids,
                              attention_mask=b_input_mask)

            # Calculate the loss.
            loss = loss_fn(logits, b_labels)

            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids)


        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Save the best model till now if we have the least loss in the current epoch.
        save_best_model(
            avg_val_loss, epoch_i, model, optimizer
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


    # ========================================
    #              Loading Model
    # ========================================

    # Load best model
    model_idx = model_i + 1
    print("")
    print(f"Loading best model for model {model_idx}")

    path = f'./drive/MyDrive/TCC/code/model_base/best_model_{model_idx}.pth'
    best_model_cp = torch.load(path)
    best_model_epoch = best_model_cp['epoch']
    best_model_valid_loss = best_model_cp['valid_loss']
    print(f"  Best model was saved at {best_model_epoch} epochs\n")

    model = CustomModel()
    model.load_state_dict(best_model_cp['model_state_dict'])

    model.to(device)


    # ========================================
    #                 Testing
    # ========================================

    print("")
    print("Running Test...")

    # Create a DataLoader to batch our test samples for us. We'll use a sequential
    # sampler this time--don't need this to be random!
    prediction_sampler = SequentialSampler(test_dataset)
    prediction_dataloader = DataLoader(test_dataset, sampler=prediction_sampler, batch_size=batch_size)

    # Put model in evaluation mode
    model.eval()

    # Tracking variables
    predictions , true_labels = [], []

    # Predict
    for batch in prediction_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)

      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch

      # Telling the model not to compute or store gradients, saving memory and
      # speeding up prediction
      with torch.no_grad():
          # Forward pass, calculate logit predictions.
          logits = model(b_input_ids,
                        attention_mask=b_input_mask)

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      # Store predictions and true labels
      predictions.append(logits)
      true_labels.append(label_ids)

    # ========================================
    #                 Metrics
    # ========================================

    # Combine the results across all batches.
    flat_predictions = np.concatenate(predictions, axis=0)

    # For each sample, pick the label (0 or 1) with the higher score.
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

    # Combine the correct labels for each batch into a single list.
    flat_true_labels = np.concatenate(true_labels, axis=0)

    # Calculate acuracy
    accuracy = accuracy_score(flat_true_labels, flat_predictions)

    # Calculate the QWK
    qwk = cohen_kappa_score(flat_true_labels, flat_predictions, weights='quadratic')

    # Calculate the RMSE
    rmse = mean_squared_error(flat_true_labels, flat_predictions, squared=False)

    # Calculate the Horizontal Discrepancy
    hd = np.sum(abs(flat_predictions - flat_true_labels) > 2) / len(flat_true_labels)

    print('  Accuracy Score: %.4f' % accuracy)
    print('  Quadratic Weighted Kappa Score: %.4f' % qwk)
    print('  Rooted Mean Squared Error Score: %.4f' % rmse)
    print('  Horizontal Discrepancy: %.4f' % hd)

    # Record metrics for this model.
    metrics.append(
        {
            'Model': model_i + 1,
            'Best Epoch': best_model_epoch,
            'Valid. Loss': best_model_valid_loss,
            'Accuracy': accuracy,
            'QWK': qwk,
            'RMSE': rmse,
            'Horizontal Discrepancy': hd
        }
    )

    # Delete variables to free memory space
    optimizer.zero_grad(set_to_none=True)
    model.to(torch.device("cpu"))
    model = None
    del model
    del b_input_ids
    del b_input_mask
    del b_labels
    del logits
    del predictions
    del true_labels
    del batch

    gc.collect()
    torch.cuda.empty_cache()



========== Model 1 / 10 ==========


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]


======== Epoch 1 / 6 ========
Training...
  Batch   100  of    288.    Elapsed: 0:00:48.
  Batch   200  of    288.    Elapsed: 0:01:33.

  Average training loss: 1.19
  Training epoch took: 0:02:13

Running Validation...
  Accuracy: 0.54
  Validation Loss: 1.11
  Validation took: 0:00:09

Best validation loss: 1.1149019377846872

Saving best model for epoch: 1


======== Epoch 2 / 6 ========
Training...
  Batch   100  of    288.    Elapsed: 0:00:45.
  Batch   200  of    288.    Elapsed: 0:01:30.

  Average training loss: 0.98
  Training epoch took: 0:02:10

Running Validation...
  Accuracy: 0.57
  Validation Loss: 1.07
  Validation took: 0:00:09

Best validation loss: 1.0660811874174303

Saving best model for epoch: 2


======== Epoch 3 / 6 ========
Training...
  Batch   100  of    288.    Elapsed: 0:00:45.
  Batch   200  of    288.    Elapsed: 0:01:30.

  Average training loss: 0.80
  Training epoch took: 0:02:10

Running Validation...
  Accuracy: 0.57
  Validation Loss: 1.12
  Valid

## Metrics Table

In [ ]:
# Create a DataFrame from our models metrics.
df_metrics = pd.DataFrame(data=metrics)

# Use the 'Model' as the row index.
df_metrics = df_metrics.set_index('Model')

# Display the table.
df_metrics

,Best Epoch,Valid. Loss,Accuracy,QWK,RMSE,Horizontal Discrepancy
Model,,,,,,
1,2,1.066081,0.598784,0.534555,0.763265,0.007092
2,1,1.129304,0.547112,0.445292,0.810268,0.012158
3,2,1.108072,0.583587,0.530191,0.820828,0.012158
4,2,1.084650,0.560284,0.545608,0.818356,0.011145
5,2,1.092180,0.580547,0.558157,0.778381,0.008105
6,2,1.051165,0.590679,0.551313,0.777730,0.008105
7,2,1.109238,0.569402,0.489470,0.825137,0.016211
8,2,1.073789,0.566363,0.557861,0.798301,0.007092
9,2,1.090437,0.562310,0.547229,0.797666,0.009119


# Final Metrics

In [ ]:
# Accuracy
mean_accuracy = df_metrics['Accuracy'].mean()
std_accuracy = df_metrics['Accuracy'].std()

# Quadratic Weighted Kappa
mean_qwk = df_metrics['QWK'].mean()
std_qwk = df_metrics['QWK'].std()

# Root Mean Score Error
mean_rmse = df_metrics['RMSE'].mean()
std_rmse = df_metrics['RMSE'].std()

# Horizontal Discrepancy
mean_hd = df_metrics['Horizontal Discrepancy'].mean()
std_hd = df_metrics['Horizontal Discrepancy'].std()

print('Mean Accuracy: ', mean_accuracy)
print('Standard Deviation Accuracy: ', std_accuracy)
print('-----------------------------------------------------------------')
print('Mean QWK: ', mean_qwk)
print('Standard Deviation QWK: ', std_qwk)
print('-----------------------------------------------------------------')
print('Mean RMSE: ', mean_rmse)
print('Standard Deviation RMSE: ', std_rmse)
print('-----------------------------------------------------------------')
print('Mean Horizontal Discrepancy: ', mean_hd)
print('Standard Deviation Horizontal Discrepancy: ', std_hd)

Mean Accuracy:  0.5745694022289768
Standard Deviation Accuracy:  0.016022847276937733
-----------------------------------------------------------------
Mean QWK:  0.5278089733676083
Standard Deviation QWK:  0.03572873976888158
-----------------------------------------------------------------
Mean RMSE:  0.7965051657201123
Standard Deviation RMSE:  0.02192439416153821
-----------------------------------------------------------------
Mean Horizontal Discrepancy:  0.0099290780141844
Standard Deviation Horizontal Discrepancy:  0.0029364491886908696
